In [6]:
from lmdb import Environment
import numpy as np
import os
import sys
import ipyvolume as ipv
import numpy as np
import ipyvolume.pylab as p3
from PythonProtocols.SceneSetup_pb2 import SceneSetup
from PythonProtocols.ScatterSample_pb2 import ScatterSample
from PythonProtocols.Result_pb2 import Result

BATCH_SIZE = 2048

class Dataset:
    env = Environment("../Data/Dataset/Train.lmdb", subdir=False, max_dbs=128, mode=0, create=False, readonly=True)
    scatter_db = env.open_db(ScatterSample.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)
    scenes_db = env.open_db(SceneSetup.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)
    results_db = env.open_db(Result.DESCRIPTOR.full_name.encode('ascii'), integerkey=True, create=False)

In [10]:
from PythonProtocols.SceneSetup_pb2 import SceneSetup

dataset = Dataset()

clouds = {}
cloudNames = []

with dataset.env.begin() as transaction:
    cursor = transaction.cursor(dataset.scenes_db)
    cursor.first()
    for key, value in cursor:
        scene = SceneSetup()
        scene.ParseFromString(value)
        if scene.cloud_path not in clouds.keys():
            clouds[scene.cloud_path] = ([],[])
            cloudNames.append(scene.cloud_path)
        id = int.from_bytes(key, byteorder='little')
        clouds[scene.cloud_path][0].append(id)
        clouds[scene.cloud_path][1].append(scene)

In [11]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from PythonProtocols.ScatterSample_pb2 import ScatterSample
from PythonProtocols.Result_pb2 import Result

cloud = widgets.Dropdown(
        options=cloudNames,
        value=cloudNames[0],
        description='Cloud:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

scene = widgets.Dropdown(
        options=clouds[cloudNames[0]][0],
        value=clouds[cloudNames[0]][0][0],
        description='Scene Id:',
        disabled=False
    )

def update_scene_options(*args):
    scene.options = clouds[cloud.value][0]
    scene.value = clouds[cloud.value][0][0]
    
cloud.observe(update_scene_options, 'value')

def update(cloud, sceneId):
    dataset = Dataset()
    
    x = np.zeros(BATCH_SIZE)
    y = np.zeros(BATCH_SIZE)
    z = np.zeros(BATCH_SIZE)

    u = np.zeros(BATCH_SIZE)
    v = np.zeros(BATCH_SIZE)
    w = np.zeros(BATCH_SIZE)

    c = np.zeros((BATCH_SIZE, 3))
    cSelected = np.zeros((BATCH_SIZE, 3))
    
    with dataset.env.begin() as transaction:
        sample = ScatterSample()
        radiance = Result()
        for i in range(BATCH_SIZE):
            id = sceneId * BATCH_SIZE + i
            result = transaction.get(id.to_bytes(4, 'little'), db=dataset.scatter_db)
            sample.ParseFromString(result)
            x[i] = sample.point.x
            y[i] = sample.point.y
            z[i] = sample.point.z

            u[i] = sample.view_direction.x
            v[i] = sample.view_direction.y
            w[i] = sample.view_direction.z
            
            result = transaction.get(id.to_bytes(4, 'little'), db=dataset.results_db)
            radiance.ParseFromString(result)
            
            color = radiance.light_intensity
            c[i, :] = [color, color * 0.8, color * 0.6]
            

        fig = ipv.figure(width=700, height=700, lighting=False)
        ipv.quiver(x, y, z, u, v, w, color=c, size=2, color_selected='red', size_selected=2)
        ipv.xyzlim(-0.5, 0.5)
        ipv.pylab.style.set_style_dark()
        ipv.pylab.style.box_off()
        ipv.pylab.style.axes_off()
        p3.selector_default()
        fig.camera_control = 'orbit'
        
        def set_angles(fig, i, fraction):
            print(i)
            fig.angley = fraction*np.pi*2
        p3.show()

w = interactive(update, cloud=cloud, sceneId=scene)
display(w)

interactive(children=(Dropdown(description='Cloud:', layout=Layout(width='50%'), options=('10_FREEBIE_CLOUDS\\…